In [ ]:
import logging
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xarray as xr
from dask.distributed import Client, LocalCluster
from hwt import firstRun
from metpy.constants import g
from tqdm.notebook import tqdm

# Place where Dave cached datasets for quicker use
CACHEDIR = Path("/glade/derecho/scratch/ahijevyc/ldmason_hwt")

now = pd.Timestamp.now()
year = 2024
init_start = firstRun(year)
init_end = pd.to_datetime(f"{year}0531")

init_times = pd.date_range(
    init_start.floor("24h"), init_end.floor("24h"), freq="24h"
)  # round down to nearest multiple

forecast_length = 168

In [ ]:
parallel = True
if parallel:
    try:
        client.close()
    except NameError:
        pass

    try:
        cluster.close()
    except NameError:
        pass
    # Create a local cluster with workers (CPUs)
    cluster = LocalCluster(threads_per_worker=1)
    client = Client(cluster)
client

In [ ]:
shortName, isobaricInhPa, units = "z", 500, "meter"


def build_file_url(init_time, mem, forecast_hour):
    return (
        f"https://nomads.ncep.noaa.gov/pub/data/nccf/com/gens/prod/gefs.{init_time:%Y%m%d}/"
        f"{init_time:%H}/atmos/pgrb2ap5/"
        f"{mem}.t{init_time:%H}z.pgrb2a.0p50.f{forecast_hour:03d}"
    )


def local_path_from_url(gefsdir, file_url_str):
    url_path = Path(file_url_str)
    file_path = gefsdir.joinpath(*url_path.parts[-5:])
    os.makedirs(file_path.parent, exist_ok=True)
    return file_path


def open_grib_dataset(path):
    ds = xr.open_dataset(
        path,
        engine="cfgrib",
        backend_kwargs={"errors": "ignore"},
        filter_by_keys={
            "typeOfLevel": "isobaricInhPa",
            "level": isobaricInhPa,
            "shortName": "gh" if shortName == "z" else shortName,
        },
        decode_timedelta=True,
        chunks={},  # chunking can help reduce memory usage
    )
    if shortName == "z":
        ds = ds.rename(gh="z")
    return ds


def download_file(url, local_file_path, delay_seconds=1):
    try:
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()
        with open(local_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Successfully downloaded: {local_file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error during download: {e}")
        print(f"Could not download {url}")


def ai_ifiles(pangu_dir, model, init_time, mem, forecast_length):
    # return list of files at different lead times.
    if init_time > pd.to_datetime("20250101"):
        file_path = (
            pangu_dir / init_time.strftime("%Y%m%d%H") / f"ens{mem}" / f"{model}_forecast_data"
        )
        ifiles = [
            file_path / f"{model}_ens{mem}_pred_{i:03d}.nc"
            for i in range(24, forecast_length + 1, 24)
        ]
    else:
        control_or_perturbation = "p" if mem > 0 else "c"
        file_path = (
            pangu_dir / init_time.strftime("%Y%m%d%H") / f"{control_or_perturbation}{mem:02d}"
        )
        ifiles = [
            file_path / f"{model}_gefs_pred_{i:03d}.nc" for i in range(24, forecast_length + 1, 24)
        ]

    return ifiles


def add_ensemble_number(ds):
    """
    A preprocessing function to be used with xr.open_mfdataset.
    It extracts the ensemble member number from the source filename
    and adds it as a 'number' coordinate.
    """
    # Get the basename of the file (e.g., "pangu_ens0_pred_162.nc")
    try:
        filename = os.path.basename(ds.encoding["source"])
    except (KeyError, TypeError):
        # Fallback if source encoding is not available
        return ds

    # Use a regular expression to find the number following 'ens'
    match = re.search(r"ens(\d+)", filename)

    if match:
        # Extract the number, convert to integer
        ensemble_number = int(match.group(1))

        # Add 'number' as a new dimension and assign the extracted number as its coordinate
        return ds.expand_dims(number=[ensemble_number])

    return ds

In [ ]:
def get_nested_files(ai_dir, init_times, forecast_length, nmem=51):
    # derive model from ai_dir
    model = ai_dir.name.rstrip("_realtime")
    # Create a nested list where list[init_time][member] = [files_for_all_forecast_hours]
    nested_files = []
    for init_time in init_times:
        # List for all members for this init_time
        time_specific_files = []
        for mem in range(nmem):
            # List of all forecast hour files for this specific member
            lead_times = ai_ifiles(ai_dir, model, init_time, mem, forecast_length)
            if all([os.path.exists(f) for f in lead_times]):
                time_specific_files.append(lead_times)
            else:
                print(f"not all lead times present for {model} {mem} {init_time}")

        if len(time_specific_files) != nmem:
            print(f"only {len(time_specific_files)}/{nmem} {model} {init_time} files")
            continue
        nested_files.append(time_specific_files)

    return nested_files


def merge_nested_files(nested_files, shortName, isobaricInhPa, units):
    # The channel label we want to select
    channel_label = f"{shortName}{isobaricInhPa}"

    nmem = len(nested_files[0])
    ds = (
        xr.open_mfdataset(
            nested_files,
            combine="nested",
            concat_dim=["init_time", "number", "prediction_timedelta"],
            chunks="auto",  # Use 'auto' for better performance with dask
        )
        # Rename dimensions and coordinates at the start
        .rename(
            {
                "init_time": "initialization_time",
                "prediction_timedelta": "step",
                "lat": "latitude",
                "lon": "longitude",
                "__xarray_dataarray_variable__": shortName,  # Rename the main variable
            }
        )
        # Assign the integer coordinate for the 'number' dimension
        .assign_coords(number=range(nmem))
        # Convert 'step' dimension from timedelta to integer forecast hours
        .pipe(
            lambda ds: ds.assign_coords(
                forecast_hour=("step", ds["step"].data / np.timedelta64(1, "h"))
            ).swap_dims({"step": "forecast_hour"})
        )
        # Calculate the valid_time coordinate
        .assign(valid_time=lambda ds: ds.initialization_time + ds.step)
        # Select the desired channel by its label (more readable)
        .sel(channel=channel_label)
        # Add the pressure level as a non-dimension coordinate
        .assign_coords(isobaricInhPa=isobaricInhPa)
    )

    if shortName == "z":
        if "units" in ds["z"].attrs:
            # Let metpy take care of the units
            ds = ds.metpy.quantify()
            print("divide by g (as Quantity with units)")
            ds["z"] /= g
            ds = ds.metpy.dequantify()
        else:
            # no units, like in fengwu
            print("divide by g (no units)")
            ds["z"] /= g.m
            ds["z"].attrs.update({"units": units})
    assert (
        ds[shortName].attrs["units"] == units
    ), f"expected units {units}. got {ds[shortName].attrs['units']}"
    return ds

In [ ]:
if init_start > pd.to_datetime("20250101"):
    # Output from real-time runs in Ryan's directory
    pangu_dir = Path("/glade/derecho/scratch/sobash/pangu_realtime")
    nested_files = get_nested_files(pangu_dir, init_times, forecast_length)
    ds_pangu = merge_nested_files(nested_files, shortName, isobaricInhPa)

    fengwu_dir = Path("/glade/derecho/scratch/sobash/fengwu_realtime")
    nested_files = get_nested_files(fengwu_dir, init_times, forecast_length)
    ds_fengwu = merge_nested_files(nested_files, shortName, isobaricInhPa)
else:
    ofile = (
        CACHEDIR / f"pangu.{init_start:%Y%m%d%H}-{init_end:%Y%m%d%H}.{forecast_length:03d}h.zarr"
    )
    if os.path.exists(ofile):
        print(f"open existing {ofile}")
        ds_pangu = xr.open_zarr(ofile)
    else:
        print(f"making {ofile}")
        # Output from older dates (HWT2023/4)
        # Nested list
        #                         init0          , ... ,           initn
        # model_runs = [[gec0, gep1, ... , gep30], ... , [gec0, gep1, ... , gep30]]
        model_runs = []
        for init_time in init_times:
            idir = Path(
                "/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather"
            ) / init_time.strftime("%Y%m%d%H")
            ifiles = sorted(list(idir.glob("g??[0-9][0-9].grib")))
            if len(ifiles) == 31:
                model_runs.append(ifiles)
            else:
                logging.warning(f"{init_time} has {len(ifiles)}/31 pangu files in {idir}")
        ds = xr.open_mfdataset(
            model_runs,
            engine="cfgrib",
            backend_kwargs={"errors": "ignore"},
            filter_by_keys={
                "typeOfLevel": "isobaricInhPa",
                "level": isobaricInhPa,
                "shortName": shortName,  # Don't worry about z being called gh. It's called z.
            },
            decode_timedelta=True,
            chunks={},  # chunking can help reduce memory usage
            combine="nested",
            concat_dim=["time", "number"],
        )
        ds = ds.rename(time="initialization_time")
        # Convert 'step' dimension from timedelta to integer forecast hours
        ds = ds.assign_coords(
            forecast_hour=("step", ds["step"].data / np.timedelta64(1, "h"))
        ).swap_dims({"step": "forecast_hour"})
        # Just multiples of 24 hours, please.
        ds = ds.sel(forecast_hour=range(0, forecast_length + 1, 24))
        if shortName == "z":
            # Let metpy take care of units (must be Quantity first)
            # m^2/s^2 -> m
            ds = ds.metpy.quantify()
            ds["z"] /= g
            ds = ds.metpy.dequantify()
        ds_pangu = ds
        ds_pangu.to_zarr(ofile)
    ofile = (
        CACHEDIR / f"fengwu.{init_start:%Y%m%d%H}-{init_end:%Y%m%d%H}.{forecast_length:03d}h.zarr"
    )
    if os.path.exists(ofile):
        print(f"open existing {ofile}")
        ds_fengwu = xr.open_zarr(ofile)
    else:
        print(f"making {ofile}")
        fengwu_dir = Path("/glade/derecho/scratch/ahijevyc/ai-models/output/fengwu")
        nested_files = get_nested_files(fengwu_dir, init_times, forecast_length, nmem=31)
        ds_fengwu = merge_nested_files(nested_files, shortName, isobaricInhPa, units)
        ds_fengwu.to_zarr(ofile)
ds_pangu

In [ ]:
gefs_members = ["gec00"] + [f"gep{i:02d}" for i in range(1, 31)]
gefsdir = Path("/glade/derecho/scratch/ahijevyc/tmp/gefs")

existing_files = set(p.resolve() for p in gefsdir.glob("*/??/atmos/pgrb2ap5/*pgrb2a.0p50.f???"))

# Base URL for the public NOAA GEFS S3 bucket
base_url = "https://noaa-gefs-pds.s3.amazonaws.com"
# ===================================================================
# PHASE 1: DEFINE FILE REQUIREMENTS
# ===================================================================
print("--- Phase 1: Defining all required file paths ---")
required_files = []

for init_time in init_times:
    for member in gefs_members:
        for fhr in range(0, forecast_length + 1, 24):
            valid_time = init_time + pd.Timedelta(hours=fhr)

            fhr_str = f"{fhr:03d}"
            s3_filename = f"{member}.t{init_time:%H}z.pgrb2a.0p50.f{fhr_str}"
            file_path_on_s3 = f"gefs.{init_time:%Y%m%d}/{init_time:%H}/atmos/pgrb2ap5/{s3_filename}"
            url = f"{base_url}/{file_path_on_s3}"
            local_path = local_path_from_url(gefsdir, url)
            required_files.append({"url": url, "local_path": local_path})

            # Now that we use ERA5 we don't need GEFS for the extra forecast valid times
            # TODO: remove references to valid_time in this loop. Then you can remove
            # method="nearest" in the plot of 
            # 0-hour fcst at valid_time so we have "truth" to compare forecasts to.
            s3_filename = f"{member}.t{valid_time:%H}z.pgrb2a.0p50.f000"
            file_path_on_s3 = (
                f"gefs.{valid_time:%Y%m%d}/{valid_time:%H}/atmos/pgrb2ap5/{s3_filename}"
            )
            url = f"{base_url}/{file_path_on_s3}"
            local_path = local_path_from_url(gefsdir, url)
            required_files.append({"url": url, "local_path": local_path})

# required files will have duplicates where an init_time is also a valid_time
# But that is okay, because if the local_path exists already the url is not downloaded
print(f"✅ Defined {len(required_files)} files required for analysis (duplicates).\n")

unique_list = []
seen = set()
for d in required_files:
    # convert dict to tuple of key-value pairs. assumes order of dict keys is consistent
    t = tuple(d.items())
    if t not in seen:
        seen.add(t)
        unique_list.append(d)
print(f"  {len(unique_list)} unique files required for analysis.\n")
required_files = unique_list

print(f"✅ Data saved in: {gefsdir}")
print("-" * 50)
datasets = []

ofile = CACHEDIR / f"gefs{init_start:%Y%m%d%H}-{init_end:%Y%m%d%H}.{forecast_length:03d}h.zarr"
# Try saved ds_gefs. I tried saving zarr with compute=False but when I loaded, values were zero.
if os.path.exists(ofile):
    print(f"open existing {ofile}")
    ds_gefs = xr.open_zarr(ofile)
else:
    print(f"making {ofile}")
    for required_file in tqdm(required_files):
        url = required_file["url"]
        local_file_path = required_file["local_path"]
        if local_file_path in existing_files:
            # print(f"   -> 🟢 File already exists. Skipping.")
            pass
        else:
            print(local_file_path)
            # Ensure the destination directory exists before downloading
            Path(local_file_path).parent.mkdir(parents=True, exist_ok=True)
            print(f"   -> ⬇️  Attempting to download: {url}")
            download_file(url, local_file_path)
        ds_gefs = open_grib_dataset(local_file_path)
        datasets.append(ds_gefs)

    ds_gefs = xr.combine_nested(datasets, concat_dim=["time"])

    itime_coords = ds_gefs["valid_time"] - ds_gefs["step"]
    ds_gefs = ds_gefs.assign_coords(initialization_time=itime_coords)
    ds_gefs = ds_gefs.groupby(["initialization_time", "step", "number"]).first()

    step_as_hours = ds_gefs["step"].data / pd.to_timedelta("1h")
    ds_gefs = ds_gefs.assign_coords(forecast_hour=("step", step_as_hours))
    ds_gefs = ds_gefs.swap_dims(step="forecast_hour")
    ds_gefs["valid_time"] = ds_gefs.initialization_time + ds_gefs.step
    ds_gefs.to_zarr(ofile)

ds_gefs

In [ ]:
# Apply the geographic selection to each dataset and re-assign it to the original variable
geographic_sel = dict(latitude=slice(60, 20), longitude=slice(220, 300))
ds_gefs = ds_gefs.sel(geographic_sel).load()
ds_pangu = ds_pangu.sel(geographic_sel).load()
ds_fengwu = ds_fengwu.sel(geographic_sel).load()

In [ ]:
# fill in forecast_hour=0 in fengwu with forecast_hour=0 in pangu
# drop "valid_time" coordinate or DTypePromotionError
# tried xr.Dataset.combine_first instead of xr.concat but had troubles
if 0 not in ds_fengwu.forecast_hour:
    ds_fengwu = xr.concat(
        [ds_pangu.sel(forecast_hour=0).reset_coords("valid_time"), ds_fengwu], dim="forecast_hour"
    )

In [ ]:
era5_dir = Path("/glade/campaign/collections/rda/data/ds633.0/e5.oper.an.pl")
freq = pd.to_timedelta("24h")
# tack on extra truth times to compare to the forecast valid times in the last forecast
extra_times = pd.date_range(
    start=init_times[-1] + freq,
    end=init_times[-1] + pd.to_timedelta(f"{forecast_length}h"),
    freq=freq,
)
era5_files = [
    era5_dir
    / i.strftime("%Y%m")
    / f"e5.oper.an.pl.128_129_{shortName}.ll025sc.{i:%Y%m%d00}_{i:%Y%m%d23}.nc"
    for i in init_times.union(extra_times)
]
truth = (
    xr.open_mfdataset(era5_files)
    .sel(level=isobaricInhPa)
    .sel(geographic_sel)  # geographic selection
    .rename(Z="z", time="valid_time", level="isobaricInhPa")
)
# era5 available every hour. just return 0z please
truth = truth.sel(valid_time=(truth.valid_time.dt.hour == 0))
if shortName == "z":
    truth["z"] = truth["z"] / g.m
truth

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(15, 5), sharey=True)

# These will store the handles and labels for shared legend
handles = []
labels = []

for ax, ds_model, title in zip(
    axes, [ds_gefs, ds_pangu, ds_fengwu], ["GEFS", "Pangu-Weather", "Fengwu"]
):
    # --- Data processing steps (explained above) ---
    ensemble_mean = ds_model.mean(dim=["number", "latitude", "longitude"])

    # --- Data Reshaping (Stacking) ---
    # To prepare for plotting against 'valid_time', we first need to reshape the
    # data. `.stack()` combines the two dimensions into a single "MultiIndex"
    # dimension named 'point'. The data is now effectively a 1D series where
    # each data point is indexed by a (initialization_time, forecast_hour) pair.
    stacked_ds = ensemble_mean.stack(point=("initialization_time", "forecast_hour"))

    # --- Swapping the Index ---
    # This is a critical step. We replace the 'point' index with a new MultiIndex
    # composed of 'initialization_time' and 'valid_time'. This aligns each
    # data point with its specific start time and valid time, preparing it for the final pivot.
    tidy_ds = stacked_ds.set_index(point=["initialization_time", "valid_time"])

    # --- Final Pivot (Unstacking) ---
    # `.unstack()` performs the final pivot. It converts the data into the ideal 2D
    # "tidy" format for plotting:
    #   - Rows are indexed by 'valid_time' (our desired x-axis).
    #   - Columns are indexed by 'initialization_time' (our desired series/hues).
    plot_ready = tidy_ds[shortName].unstack("point")

    # --- Plotting ---
    plot_ready.plot.line(
        x="valid_time",
        hue="initialization_time",
        ax=ax,
        add_legend=False,
    )
    truth[shortName].mean(dim=["latitude", "longitude"]).plot.line(
        ax=ax, x="valid_time", marker="o", color="k"
    )

    # --- Axis formatting ---
    ax.set_title(f"{title} Forecast {isobaricInhPa}-hPa {shortName}")
    ax.set_xlabel("Valid Time")
    ax.set_ylabel(f"{shortName} [{units}]")
    ax.grid(True, linestyle="--", alpha=0.6)

    # --- Capture handles and create labels ONCE from the first plot ---
    if not handles:  # An empty list is False, so this runs only on the first iteration
        handles = ax.get_lines()
        # Create nicely formatted labels from the coordinate values
        labels = [
            pd.to_datetime(t).strftime("%Y-%m-%d %Hz")
            for t in plot_ready.initialization_time.values
        ]

# Add a shared legend to the right of the figure
fig.legend(
    handles, labels, title="Initialization Time", loc="center right", bbox_to_anchor=(1.0, 0.5)
)

# Adjust layout to make room for the legend
fig.tight_layout(rect=[0, 0, 0.88, 1])  # Adjusted rect to give legend more space
#ax.set_xlim((19850.65, 19862.35))

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)

# These will store the handles and labels for shared legend
handles = []
labels = []

for ax, ds_model, title in zip(
    axes, [ds_gefs, ds_pangu, ds_fengwu], ["GEFS", "Pangu-Weather", "Fengwu"]
):
    # Average over spatial dimensions ONLY
    # This keeps the 'number' dimension for the ensemble members.
    ds_processed = ds_model.mean(dim=["latitude", "longitude"])

    # Loop over each initialization time
    for init_time in ds_processed.initialization_time:
        # no spaghetti lines for run after last init_time
        if init_time > init_times[-1]:
            continue
        # Select all members for this single forecast run
        run_with_members = ds_processed.sel(initialization_time=init_time)

        # Plot the first member to establish the color and label for the legend
        first_member_data = run_with_members.isel(number=0)
        line = ax.plot(
            first_member_data.valid_time,
            first_member_data[shortName],
            alpha=0.5,
            label=pd.to_datetime(init_time.values).strftime("%Y-%m-%d %hz"),
        )
        # Get the color matplotlib automatically assigned to the first line
        run_color = line[0].get_color()

        # Loop over the REST of the members and plot them with the same color
        for member_index in ds_model.number[1:]:
            member_data = run_with_members.sel(number=member_index)
            ax.plot(
                member_data.valid_time,
                member_data[shortName],
                alpha=0.5,
                color=run_color,  # Reuse the color from the first member
            )
    truth[shortName].mean(dim=["latitude", "longitude"]).plot.line(
        ax=ax, x="valid_time", marker="o", color="k"
    )

    ax.set_title(f"{title} Forecast {isobaricInhPa}-hPa {shortName}")
    ax.set_xlabel("Valid Time")
    ax.set_ylabel(f"{shortName} [{units}]")
    ax.grid(True, linestyle="--", alpha=0.6)
    # --- Capture handles and create labels ONCE from the first plot ---
    if not handles:  # An empty list is False, so this runs only on the first iteration
        handles = ax.get_lines()[::31]
        # Create nicely formatted labels from the coordinate values
        labels = [
            pd.to_datetime(t).strftime("%Y-%m-%d %H:%M")
            for t in plot_ready.initialization_time.values
        ]

# Add a shared legend to the right of the figure
fig.legend(
    handles, labels, title="Initialization Time", loc="center right", bbox_to_anchor=(1.0, 0.5)
)

# Adjust layout to make room for the legend
fig.tight_layout(rect=[0, 0, 0.88, 1])  # Adjusted rect to give legend more space

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)

for ax, ds_model, title in zip(
    axes, [ds_gefs, ds_pangu, ds_fengwu], ["GEFS", "Pangu-Weather", "Fengwu"]
):
    ds_model = ds_model.assign_coords(day=ds_model.forecast_hour / 24)
    # average over initialization_time, lat and lon. std over ensemble (number).
    ensemble_spread = (
        ds_model[shortName]
        .std(dim="number", ddof=1)
        .mean(dim=["initialization_time", "latitude", "longitude"])
    )
    # error = ensemble mean - truth
    # Line up truth and forecast along `valid_time`.
    error = ds_model.mean(dim="number") - truth.sel(valid_time=ds_model["valid_time"], method="nearest")
    se = error[shortName] ** 2
    mse = se.mean(dim=["latitude", "longitude"])
    rmse = np.sqrt(mse).mean(dim="initialization_time")
    rmse.plot.line(ax=ax, x="day", marker="o", label="rmse")
    ensemble_spread.plot.line(ax=ax, x="day", marker="o", label="spread")
    ax.grid(True, linestyle="--", alpha=0.6)
    ax.set_title(f"{title} Forecast {isobaricInhPa}-hPa {shortName}")
    ax.legend()
    ax.set_ylabel(f"{shortName} [{units}]")

In [ ]:
ds_pangu[shortName].plot.hist()

In [ ]:
ds_pangu[shortName].mean()

In [ ]:
ds_pangu[shortName].min(), ds_pangu[shortName].max()

In [ ]:
z = ds_pangu["z"]
# --- Please run this exact block of code with your DataArray 'z' ---

# 1. Print the full representation of your object 'z'
# This will show its name, dimensions, coordinates, and attributes.
print("--- Full representation of the DataArray 'z' ---")
print(z)
print("-------------------------------------------------")


# 2. Calculate all three statistics back-to-back
min_val = z.min()
max_val = z.max()
mean_val = z.mean()

# 3. Print the results
print(f"Min value from .min(): {min_val.item()}")
print(f"Max value from .max(): {max_val.item()}")
print(f"Mean value from .mean(): {mean_val.item()}")


# 4. Perform the crucial consistency check
is_consistent = min_val <= mean_val <= max_val

print("\n--- Consistency Check ---")
print(f"Is the result mathematically consistent? {is_consistent}")
print("-------------------------")

In [ ]:
# This forces a true NumPy calculation on the in-memory data,
# bypassing any xarray/cfgrib backend magic.
true_mean = z.values.mean()

print(f"The TRUE mathematical mean is: {true_mean}")

In [ ]:
z.attrs

In [ ]:
# del(z.attrs['GRIB_paramId'])
# del(z.attrs['GRIB_cfName'])
# del(z.attrs['GRIB_cfVarName'])
# del(z.attrs['GRIB_name'])
# del(z.attrs['GRIB_shortName'])
z.attrs.update({"long_name": "Geopotential Height", "standard_name": "geopotential height"})
z.mean()

In [ ]:
ds_fengwu[shortName].plot.hist()

In [ ]:
ds_gefs[shortName].plot.hist()

In [ ]:
ds_pangu.sel(forecast_hour=0)

In [ ]:
ds_fengwu

In [ ]:
ds_pangu